# Lab 6

Advanced APIs for Scale - Batching

In [1]:
from dotenv import load_dotenv
from openai import OpenAI
load_dotenv(override=True)

openai = OpenAI()


In [2]:
support_tickets = [
    "I can't access my account after the password reset",
    "When will the new mobile app features be released?", 
    "My credit card was charged twice for the same subscription",
    "The API is returning 500 errors for all my requests",
    "I'd like to request a bulk pricing discount for my team",
    "How do I export my data before canceling my account?",
    "The dashboard is loading very slowly since yesterday",
    "Can you add support for OAuth authentication?",
    "I was charged for premium but still see ads",
    "Why is my API rate limit so low compared to the documentation?"
]


## First step - Generate a jsonl file - json line - good for streaming and batch

With this format:

```jsonl
{"custom_id": "request-1", "method": "POST", "url": "/v1/chat/completions", "body": {"model": "gpt-3.5-turbo-0125", "messages": [{"role": "system", "content": "You are a helpful assistant."},{"role": "user", "content": "Hello world!"}],"max_tokens": 1000}}
{"custom_id": "request-2", "method": "POST", "url": "/v1/chat/completions", "body": {"model": "gpt-3.5-turbo-0125", "messages": [{"role": "system", "content": "You are an unhelpful assistant."},{"role": "user", "content": "Hello world!"}],"max_tokens": 1000}}
```

In [7]:
def make_classification_line(index):
    ticket = support_tickets[index]
    prompt = f"""Classify this support ticket into one of these categories: billing, technical, feature_request, account_access, or general_inquiry.

Ticket: {ticket}

Respond with just the category name."""
    
    # Properly escape quotes in the JSON string
    escaped_prompt = prompt.replace('"', '\\"').replace('\n', '\\n')
    
    return f'{{"custom_id": "ticket-{index}", "method": "POST", "url": "/v1/chat/completions", "body": {{"model": "gpt-4o-mini", "messages": [{{"role": "user", "content": "{escaped_prompt}"}}], "max_tokens": 50}}}}'

make_classification_line(0)

'{"custom_id": "ticket-0", "method": "POST", "url": "/v1/chat/completions", "body": {"model": "gpt-4o-mini", "messages": [{"role": "user", "content": "Classify this support ticket into one of these categories: billing, technical, feature_request, account_access, or general_inquiry.\\n\\nTicket: I can\'t access my account after the password reset\\n\\nRespond with just the category name."}], "max_tokens": 50}}'

In [10]:
# Create classification batch
with open("ticket_classification.jsonl", "w", encoding="utf-8") as f:
    for i in range(len(support_tickets)):
        f.write(make_classification_line(i) + "\n")

## Second step - upload the file

In [11]:
tickets_file = openai.files.create(
    file=open("ticket_classification.jsonl", "rb"), 
    purpose="batch"
)

## Third step - make the batch

In [63]:
#completion_window='24h' means openai you hvae 24h to do it when you want.

batch = openai.batches.create(
    input_file_id=tickets_file.id,
    endpoint="/v1/chat/completions", 
    completion_window="24h"
)
batch

Batch(id='batch_685ebed434d48190ad8a35bd9a9605ae', completion_window='24h', created_at=1751039700, endpoint='/v1/chat/completions', input_file_id='file-V7dK92MpAj5ukRw1KxnmRP', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1751126100, failed_at=None, finalizing_at=None, in_progress_at=None, metadata=None, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))

## Fourth step - monitor the batch

In [76]:
latest = openai.batches.retrieve(batch.id)
print(latest)
print(latest.status)
#Batch(id='batch_685eaeb9b3d88190bb6d360946412e5c', completion_window='24h', created_at=1751035577, endpoint='/v1/chat/completions', input_file_id='file-V7dK92MpAj5ukRw1KxnmRP', object='batch', status='in_progress', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1751121977, failed_at=None, finalizing_at=None, in_progress_at=1751035579, metadata=None, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=10))
#in_progress

Batch(id='batch_685ebed434d48190ad8a35bd9a9605ae', completion_window='24h', created_at=1751039700, endpoint='/v1/chat/completions', input_file_id='file-V7dK92MpAj5ukRw1KxnmRP', object='batch', status='in_progress', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1751126100, failed_at=None, finalizing_at=None, in_progress_at=1751039701, metadata=None, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=10))
in_progress


## Fifth step - collect the results

In [73]:
results_file_id = latest.output_file_id
results_file = openai.files.content(results_file_id)
results_file.text

'{"id": "batch_req_685ebdebdd888190926935e456b9b125", "custom_id": "ticket-0", "response": {"status_code": 200, "request_id": "da2a5c3bfb698af4d8c9d72c5f895bbf", "body": {"id": "chatcmpl-Bn5dlCn6KBXL3Ki08hW475OaLvGq0", "object": "chat.completion", "created": 1751039453, "model": "gpt-4o-mini-2024-07-18", "choices": [{"index": 0, "message": {"role": "assistant", "content": "account_access", "refusal": null, "annotations": []}, "logprobs": null, "finish_reason": "stop"}], "usage": {"prompt_tokens": 52, "completion_tokens": 2, "total_tokens": 54, "prompt_tokens_details": {"cached_tokens": 0, "audio_tokens": 0}, "completion_tokens_details": {"reasoning_tokens": 0, "audio_tokens": 0, "accepted_prediction_tokens": 0, "rejected_prediction_tokens": 0}}, "service_tier": "default", "system_fingerprint": "fp_34a54ae93c"}}, "error": null}\n{"id": "batch_req_685ebdebe9a08190b60c8ab6dc490c2a", "custom_id": "ticket-1", "response": {"status_code": 200, "request_id": "6487ad5cc8fbcc8b451026e991e67706",

## And now let's see

In [74]:
import json
lines = results_file.text.split("\n")
results = [json.loads(line) for line in lines if line]
print(f"Number of results: {len(results)}")
print("First result:")
results[0]

Number of results: 10
First result:


{'id': 'batch_req_685ebdebdd888190926935e456b9b125',
 'custom_id': 'ticket-0',
 'response': {'status_code': 200,
  'request_id': 'da2a5c3bfb698af4d8c9d72c5f895bbf',
  'body': {'id': 'chatcmpl-Bn5dlCn6KBXL3Ki08hW475OaLvGq0',
   'object': 'chat.completion',
   'created': 1751039453,
   'model': 'gpt-4o-mini-2024-07-18',
   'choices': [{'index': 0,
     'message': {'role': 'assistant',
      'content': 'account_access',
      'refusal': None,
      'annotations': []},
     'logprobs': None,
     'finish_reason': 'stop'}],
   'usage': {'prompt_tokens': 52,
    'completion_tokens': 2,
    'total_tokens': 54,
    'prompt_tokens_details': {'cached_tokens': 0, 'audio_tokens': 0},
    'completion_tokens_details': {'reasoning_tokens': 0,
     'audio_tokens': 0,
     'accepted_prediction_tokens': 0,
     'rejected_prediction_tokens': 0}},
   'service_tier': 'default',
   'system_fingerprint': 'fp_34a54ae93c'}},
 'error': None}

In [75]:
for result in results:
    id = result["custom_id"]
    question = support_tickets[int(id.split("-")[1])]
    print(question)
    print(result["response"]["body"]["choices"][0]["message"]["content"])
    print("-"*100)

I can't access my account after the password reset
account_access
----------------------------------------------------------------------------------------------------
When will the new mobile app features be released?
feature_request
----------------------------------------------------------------------------------------------------
My credit card was charged twice for the same subscription
billing
----------------------------------------------------------------------------------------------------
The API is returning 500 errors for all my requests
technical
----------------------------------------------------------------------------------------------------
I'd like to request a bulk pricing discount for my team
billing
----------------------------------------------------------------------------------------------------
How do I export my data before canceling my account?
account_access
----------------------------------------------------------------------------------------------------


## Anthropic has a similar Batch API

https://docs.anthropic.com/en/docs/build-with-claude/batch-processing